In [99]:
import json
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif,  chi2
from sklearn import svm
from scipy.sparse import csr_matrix


In [100]:
dom1 = []
dom2 = []
test_set = []
with open('data/domain1_train.json', 'r') as file:
    for line in file:
        dom1.append(json.loads(line))
        
with open('data/domain2_train.json', 'r') as file:
    for line in file:
        dom2.append(json.loads(line))
        
sam = pd.read_csv("data/sample.csv")

with open('data/test_set.json', 'r') as file:
    for line in file:
        test_set.append(json.loads(line))
        
dom1 = pd.DataFrame.from_dict(dom1)
dom2 = pd.DataFrame.from_dict(dom2)
dom2 = dom2[dom2['text'].apply(len) > 0]
test_set = pd.DataFrame.from_dict(test_set)
        

In [91]:
print(len(dom1))
print(len(dom2))
print(len(sam))
print(len(test_set))

19500
14899
1000
1000


In [92]:
dom1['label'].value_counts()
print(len(dom2.loc[dom2['label']==0]))
print(len(dom2.loc[dom2['label']==1]))

12750
2149


In [101]:

data = pd.concat([dom1, dom2], ignore_index=True)


In [104]:
from sklearn.model_selection import train_test_split
X1 = dom1[['text']] 
y1 = dom1['label']
X1, X2, y3, y4 = train_test_split(X1, y1, test_size=0.5, stratify=data[['label']])

ValueError: Found input variables with inconsistent numbers of samples: [19500, 34399]

In [94]:
from sklearn.model_selection import train_test_split
X = data[['text']] 
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=data[['label']], random_state=42)

In [95]:
y_train

6821     1
13753    0
12520    0
22169    0
20114    1
        ..
7991     1
30198    0
18830    0
5875     1
6702     1
Name: label, Length: 27519, dtype: int64

In [96]:
print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))


label
0    0.654094
1    0.345906
Name: proportion, dtype: float64
label
0    0.65407
1    0.34593
Name: proportion, dtype: float64


In [97]:
from gensim.models import Word2Vec
from sklearn.svm import SVC
import numpy as np


X = X_train['text'].tolist()
X1 =X_test['text'].tolist()
max_length = max([len(seq) for seq in X])
word2vec = Word2Vec(X, vector_size=100, window=5, min_count=1, workers=4)

def get_text_vector(text, word2vec_model):
    vector_list = [word2vec_model.wv[word] for word in text if word in word2vec_model.wv.index_to_key]
    if len(vector_list) == 0:
        return np.zeros(word2vec.vector_size)
    return np.mean(vector_list, axis=0)

X_train = np.array([get_text_vector(text, word2vec) for text in X])
X_test = np.array([get_text_vector(text, word2vec) for text in X1])

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from gensim.models import Word2Vec
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks



smote = SMOTE(sampling_strategy=0.3)
tomek = TomekLinks(sampling_strategy='majority')


pipeline = Pipeline(steps=[('o', smote), ('u', tomek)])


X_train, y_train = pipeline.fit_resample(X_train, y_train)


ValueError: The specified ratio required to remove samples from the minority class while trying to generate new samples. Please increase the ratio.

In [ ]:
print(y_train.value_counts(normalize=True))

label
0    0.654094
1    0.345906
Name: proportion, dtype: float64


In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)

c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
X2 =test_set['text'].tolist()
test_set = np.array([get_text_vector(text, word2vec) for text in X2])
test_pre = lr.predict(test_set)
result = pd.DataFrame({
    'class': test_pre,
})
result['id'] = result.index
result = result[['id'] + [col for col in result.columns if col != 'id']]
result

,id,class
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
995,995,0
996,996,1
997,997,0
998,998,1


In [ ]:
result.to_csv('result2.csv', index=False)
